In [1]:
import pythonnet

In [2]:
import pandas as pd
import subprocess
import json
import csv
import numpy as np
import time
import matplotlib
from matplotlib import pyplot as plt
import json
import sys
from typing import List, Union, Dict

In [3]:
import os
os.environ["DOTNET_ROOT"] = r"C:\Program Files\dotnet"


Магия для того, чтобы clr импортнулась

In [4]:
from clr_loader import get_coreclr
from pythonnet import set_runtime
config_file_path: str = r"C:\Users\Maksim\Obfuscation\Tokenizer\pspython.runtimeconfig.json"
rt = get_coreclr(runtime_config=config_file_path)
set_runtime(rt)

In [5]:
pythonnet.get_runtime_info()

RuntimeInfo(kind='CoreCLR', version='<undefined>', initialized=True, shutdown=False)

Подгрузка библиотек DLL

In [6]:
import clr 
import System 
# from System import Environment
# from System import Reflection

psHome = r"C:\Program Files\PowerShell\7\\"
mmi = psHome + r'Microsoft.Management.Infrastructure.dll'
clr.AddReference(mmi)
# from Microsoft.Management.Infrastructure import *

full_filename = psHome + r"System.Management.Automation.dll"
clr.AddReference(full_filename)
# from System.Management.Automation import *
# from System.Management.Automation.Language import Parser

In [7]:
sys.path.append(r"C:\Users\Maksim\source\repos\PSFeatureExtractorLibrary\PSFeatureExtractorLibrary\bin\Debug\net6.0")
clr.AddReference("PSFeatureExtractorLibrary")
import PSFeatureExtractorLibrary

Захардкоженные переменные для скорости работы токенизатора c#

In [8]:
methods_list = [PSFeatureExtractorLibrary.GroupedArrayElementRangeCounts.AnalyzeAst,
                PSFeatureExtractorLibrary.ArrayElementMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.GroupedAssignmentStatements.AnalyzeAst,
                PSFeatureExtractorLibrary.GroupedAstTypes.AnalyzeAst,
                PSFeatureExtractorLibrary.GroupedBinaryExpressionOperators.AnalyzeAst,
                PSFeatureExtractorLibrary.CmdletMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.CommandParameterNameMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.CommentMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.ConvertExpressionMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.FunctionNameMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.IntegerAndDoubleMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.InvocationOperatorInvokedObjectMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.LineByLineMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.MemberArgumentMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.MemberMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.StringMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.TypeConstraintMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.TypeExpressionMetrics.AnalyzeAst,
                PSFeatureExtractorLibrary.GroupedUnaryExpressionOperators.AnalyzeAst,
                PSFeatureExtractorLibrary.VariableNameMetrics.AnalyzeAst
]

length_list = [22, 312, 14, 152, 104, 312, 312, 312, 312, 312, 312, 312, 312, 312, 312, 312, 312, 312, 26, 312]
length_cumulative_list = np.cumsum(length_list)

In [9]:
def tokenizer_CS(command: str, methods_list, length_cumulative_list) -> List[float]:
    ast, token, parseerror = System.Management.Automation.Language.Parser.ParseInput(command)
    
    if (len(parseerror)!=0):
        return False
    
    total_features: int = length_cumulative_list[-1]
    res_list = np.zeros(total_features, dtype="float64")
    
    first_index = 0
    last_index = length_cumulative_list[0]
    
    for i in range(len(methods_list)):
        last_index = length_cumulative_list[i]
        command_handling_result = methods_list[i](ast)
        res_list[first_index:last_index] = list(command_handling_result.Values)
        first_index = last_index

    return list(res_list)

In [10]:
def tokenizer_CS_with_features(command: str, methods_list, length_cumulative_list) -> Dict[str, List[Union[str, int, float]]]:
    ast, token, parseerror = System.Management.Automation.Language.Parser.ParseInput(command)
    
    if (len(parseerror)!=0):
        return False
    
    total_features: int = length_cumulative_list[-1]
    res_list = np.zeros(total_features, dtype="float64")
    feature_names = [''] * total_features
    
    first_index = 0
    last_index = length_cumulative_list[0]
    
    for i in range(len(methods_list)):
        last_index = length_cumulative_list[i]
        command_handling_result = methods_list[i](ast)
        res_list[first_index:last_index] = list(command_handling_result.Values)
        feature_names[first_index:last_index] = list(command_handling_result.Keys)            
        first_index = last_index

    return {"result_keys": feature_names, "result_values": list(res_list)}

Токенизатор PS

In [21]:
def tokenizer_PS(command: str):
    script_feature = f'Get-RvoFeatureVector -ScriptBlock {{{command}}}'
    res_feature = subprocess.run(["pwsh", "-Command", script_feature], capture_output=True)
    output_bytes = res_feature.stdout
    output_str = output_bytes.decode("utf-8")
    try:
        output_dict = json.loads(output_str)
    except ValueError as e:
        return False
    
    return list(output_dict.values())

In [13]:
command = "cmd   /c   ""set  tic=Get-ChildItem -path HKLM:\SYSTEM\CurrentControlSet\Services\SNMP -Recurse&&C:\WINDOWS\system32\mshta  VBScript:CreateObject(""WScript.Shell"").Run(""powershell      ^& ( '{0}{3}{1}{2}{4}{5}' -f'I','k','e-','nvo','Expressio','n' ) ( ( .('GI'  ) ( '{1}{0}'-f 'c','env:ti') ).'Value' )"",(23-1-21),True)(Window.Close)"""
# res_PS = tokenizer_PS(command)
res_CS = tokenizer_CS(command, methods_list, length_cumulative_list)
print(res_CS)

False


In [14]:
print(csv.field_size_limit())
print(sys.maxsize)

131072
9223372036854775807


In [15]:
def set_field_size_limit() -> None:
    maxInt = sys.maxsize
    while True:
        # decrease the maxInt value by factor 10 
        # as long as the OverflowError occurs.
        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)
            
        
set_field_size_limit()

In [16]:
def get_data(filename: str) -> List[str]:
    with open(filename, encoding="UTF8") as file:
        reader = csv.reader(file)
        for row in reader:
            yield row

In [17]:
def write_csv(filename: str, row: List[Union[str, int, float]]) -> None:
    with open(filename, mode='a', newline='', encoding="UTF8") as file:
        writer = csv.writer(file)
        writer.writerow(row)

In [18]:
def tokenize_data(input_filename: str, output_filename: str, wrong_command_filename: str, required_columns_indexes: List[int] = None) -> int:
    generator = get_data(input_filename)
    all_column_names: List[str] = next(generator)
    
    if required_columns_indexes is None:
        required_columns_indexes = list(range(len(all_column_names)))
    required_columns: List[str] = [all_column_names[i] for i in required_columns_indexes]  
    # ['command_clear', 'command_obfuscated', 'obfuscated']
    
    row_number: int = 2
    is_added_header: bool = False
    proccessed_values: List[float]
        
    while True:
        if not row_number % 500: 
            print(f"Row №{row_number} is proccessed.\n")
        row: List[str] = next(generator, None)
        if row is None:
            print("Dataset proccessing is completed succesfully!")
            break

        clear_command, obfuscated_command, target_value = [row[i] for i in required_columns_indexes]
        target_value = int(target_value)

        proccessing_result = tokenizer_CS(obfuscated_command, methods_list, length_cumulative_list) if is_added_header else tokenizer_CS_with_features(obfuscated_command, methods_list, length_cumulative_list)

        if not proccessing_result:
            values = [row_number, clear_command, obfuscated_command, target_value]
            write_csv(filename=wrong_command_filename, row=values)
            row_number += 1
            continue

        if not is_added_header:  # inserting header
            features, proccessed_values = proccessing_result["result_keys"], proccessing_result["result_values"]
            output_header: List[str] = required_columns[:-1] + features + [required_columns[-1]]
            write_csv(filename=output_filename, row=output_header)
            is_added_header = True
        else:
            proccessed_values = proccessing_result

        row_values: List[Union[str, int, float]] = [clear_command, obfuscated_command] + proccessed_values + [target_value]
        write_csv(filename=output_filename, row=row_values)
        row_number += 1
    return 0

### Processing sourse powershell dataset 

In [19]:
INPUT_FILENAME: str = r"D:\Obfuscation\data\datasets\POWERSHELL_DATASET.csv"
OUTPUT_FILENAME: str = r"D:\Obfuscation\data\datasets\TOKENIZER_DATASET.csv"
WRONG_COMMAND_FILENAME: str = r"D:\Obfuscation\data\datasets\WRONG_COMMANDS.csv"

In [40]:
tokenize_data(
    input_filename=INPUT_FILENAME,
    required_columns_indexes=[0, 1, 7],
    output_filename=OUTPUT_FILENAME,
    wrong_command_filename=WRONG_COMMAND_FILENAME,
)

Row №500 is proccessed.

Row №1000 is proccessed.

Row №1500 is proccessed.

Row №2000 is proccessed.

Row №2500 is proccessed.

Row №3000 is proccessed.

Row №3500 is proccessed.

Row №4000 is proccessed.

Row №4500 is proccessed.

Row №5000 is proccessed.

Row №5500 is proccessed.

Row №6000 is proccessed.

Row №6500 is proccessed.

Row №7000 is proccessed.

Row №7500 is proccessed.

Row №8000 is proccessed.

Row №8500 is proccessed.

Row №9000 is proccessed.

Row №9500 is proccessed.

Row №10000 is proccessed.

Row №10500 is proccessed.

Row №11000 is proccessed.

Row №11500 is proccessed.

Row №12000 is proccessed.

Row №12500 is proccessed.

Row №13000 is proccessed.

Row №13500 is proccessed.

Row №14000 is proccessed.

Row №14500 is proccessed.

Row №15000 is proccessed.

Row №15500 is proccessed.

Row №16000 is proccessed.

Row №16500 is proccessed.

Row №17000 is proccessed.

Row №17500 is proccessed.

Row №18000 is proccessed.

Row №18500 is proccessed.

Row №19000 is procces

### Processing poor obfuscated dataset

In [29]:
INPUT_FILENAME: str = r"D:/Obfuscation/data/datasets/additional/data/POOR_OBFUSCATED_DATASET.csv"
OUTPUT_FILENAME: str = r"D:/Obfuscation/data/datasets/additional/data/POOR_OBFUSCATED_TOKENIZER_DATASET.csv"
WRONG_COMMAND_FILENAME: str = r"D:/Obfuscation/data/datasets/additional/data/POOR_OBFUSCATED_WRONG_COMMANDS.csv"

In [32]:
tokenize_data(
    input_filename=INPUT_FILENAME,
    output_filename=OUTPUT_FILENAME,
    wrong_command_filename=WRONG_COMMAND_FILENAME,
)

Row №500 is proccessed.

Row №1000 is proccessed.

Row №1500 is proccessed.

Dataset proccessing is completed succesfully!


0

### Processing dataset containing commands with cyrillic symbols

In [21]:
INPUT_FILENAME: str = r"D:/Obfuscation/data/datasets/additional/data/CYRILLIC_DATASET.csv"
OUTPUT_FILENAME: str = r"D:/Obfuscation/data/datasets/additional/data/CYRILLIC_TOKENIZER_DATASET.csv"
WRONG_COMMAND_FILENAME: str = r"D:/Obfuscation/data/datasets/additional/data/CYRILLIC_WRONG_COMMANDS.csv"

In [22]:
tokenize_data(
    input_filename=INPUT_FILENAME,
    output_filename=OUTPUT_FILENAME,
    wrong_command_filename=WRONG_COMMAND_FILENAME,
)

Dataset proccessing is completed succesfully!


0